# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName",engine)
df
X = df.message
y = df.iloc[:,4:].values

In [6]:
print(X.shape)
print(y.shape)
print(X[0])
print(y[0])

(26216,)
(26216, 36)
Weather update - a cold front from Cuba that could pass over Haiti
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 2. Write a tokenization function to process your text data

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9 ]","",text.lower().strip())
#     print(text)
    tokens = word_tokenize(text)
#     print(tokens)
    tokens2 = [lemmatizer.lemmatize(stemmer.stem(word)) for word in tokens if word not in stop_words]
    return tokens2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Test
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

rf = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ("clf",rf)     
])

In [10]:
print(pipeline.get_params)

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
# y_train

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score

y_pred = pipeline.predict(X_test)

m = MultiLabelBinarizer().fit(y_test)

f1 = f1_score(m.transform(y_test),
         m.transform(y_pred),
         average='macro')
print(f1)
precision = precision_score(m.transform(y_test),
         m.transform(y_pred),
         average='macro')
print(precision)

recall = recall_score(m.transform(y_test),
         m.transform(y_pred),
         average='macro')
print(recall)

0.761687579466
0.755552074631
0.770189240056



### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = [
    {"tfidf__use_idf" : [True,False],
    "clf__estimator__n_estimators" : [10, 30,100,300,1000]}
]

cv = GridSearchCV(pipeline, parameters,cv=3, n_jobs=-1,verbose=3)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.2268843454379005, total=  45.3s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   53.2s remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.22993591699725358, total=  44.9s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.23481843149221848, total=  45.8s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, score=0.2354287458040891, total=  45.4s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, score=0.2311565456209948, total=  46.4s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, score=0.23329264571254196, total=  46.8s
[CV] clf__estimator__n_estimators=30, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=30, tfidf__use_idf=True, score=0.2511443393347574, total= 2.1min
[CV] clf__estimator__n_estimators=30, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=30, tfidf__use_idf=True, score=0.24580408910588952, total= 2.1min
[CV] clf__estimator__n_estimators=30, t

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(cv.best_params_)
y_pred = cv.predict(X_test)

m = MultiLabelBinarizer().fit(y_test)

f1 = f1_score(m.transform(y_test),
         m.transform(y_pred),
         average='macro')
print(f1)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.